In [15]:
import Atlas
import random
import pandas as pd

In [16]:
try:
    tape = Atlas.Tape()
except:
    pass

### Market Setup
First, we create a curve. As the time of writing, the only available curve is the ```ZeroRateCurveStrategy```. We create random rates and register them in the tape.

In [17]:
## Create Market Data
evalDate = Atlas.Date(1, Atlas.August, 2020)
store = Atlas.MarketStore(evalDate, Atlas.USD()) # store with USD as base currency

# create random rates
dt = 0.5
rates = []
dates = []
startRate = 0.01
for i in range(25):
    rate = Atlas.Dual(startRate + random.random()*0.01)
    tape.registerInput(rate) # we need to register the input to the tape for later use
    rates.append(rate)
    dates.append(evalDate + int(i*180))

tape.newRecording() # start recording, for later use

# define curve
curveDayCounter = Atlas.Actual360()
curveCompounding = Atlas.Simple
curveFrequency = Atlas.Annual

strategy = Atlas.ZeroRateLinearStrategy(dates, rates)
curve = Atlas.YieldTermStructure(strategy)
index = Atlas.RateIndex(evalDate, curveFrequency, curveDayCounter, curveFrequency, curveCompounding)
store.addCurve("USD", curve, index)

### Portfolio Setup

We create a random porfolio of ```FixFloatSwap```, with long and short positions. For simplicity, all trades start today, otherwise we would need to add fixing rates inside the ```RateIndex``` object -a must in a more realistic scenario-.

In [18]:
## set seed
random.seed(1234)
def generateInstruments(numInstruments):
    instruments = []
    for i in range(numInstruments):
        # Generate random start and end dates
        startYear = evalDate.year() # for simplicity, we will only generate instruments starting today (to avoid fixings)
        endYear = random.randint(startYear+1, 2030)
        startDate = Atlas.Date(1, Atlas.August, startYear)
        endDate = Atlas.Date(1, Atlas.August, endYear)
        
        # Create the instrument with random parameters
        rateValue = Atlas.Dual(random.uniform(0.01, 0.1))
        dayCounter = Atlas.Thirty360()
        compounding = Atlas.Simple
        frequency = Atlas.Annual
        rate = Atlas.InterestRate(rateValue, dayCounter, compounding, frequency)
        discountContext = store.curveContext("USD")
        rateIndexContext = store.rateIndexContext("USD")
        spread = Atlas.Dual(0.0)
        notional = random.uniform(100000, 1000000)
        paymentFrequency = Atlas.Quarterly
        # random side
        if random.random() > 0.5:
            side = Atlas.Payer
        else:
            side = Atlas.Receiver
        instrument = Atlas.FixFloatSwap(startDate, endDate, notional, rate, spread, paymentFrequency, rateIndexContext, side)
        instrument.firstLeg().discountCurveContext(discountContext)
        instrument.secondLeg().discountCurveContext(discountContext)
        # Add the instrument to the list
        instruments.append(instrument)
    return instruments

instruments = generateInstruments(1000)

### Indexing and Pricing

As always, we index the instruments, produce the market data points and price each instrument.

In [19]:
#indexing
indexer = Atlas.Indexer()
for inst in instruments:
    indexer.visit(inst)
request = indexer.request()

#market data simulation
model = Atlas.SpotMarketDataModel(request, store)
marketData = model.marketData(evalDate)

We calculate the NPV of the portfolio.

In [20]:
npv = Atlas.Dual(0.0)
tape.registerOutput(npv)
npvCalculator = Atlas.NPVCalculator(marketData)

for inst in instruments:
    npvCalculator.visit(inst)

npv = npvCalculator.results()
print("NPV: ", npv)

NPV:  Dual(-2447403.778020)


### Sensitivities

We calculate the sensitivities using the tape. First we seed and then compute the adjoints. Sensitivies to each rate are recovered by the ```getDerivatives``` method.

In [21]:
npv.setDerivative(1)
tape.computeAdjoints()

sens = {}
for date, rate in zip(dates, rates):
    sens[date] = round(rate.getDerivative()*0.0001, 4)


df = pd.DataFrame.from_dict(sens, orient='index', columns=['Sensitivity'])
df.index.name = 'Date'
df

,Sensitivity
Date,
01-08-2020,2.1934
28-01-2021,17.2961
27-07-2021,129.6833
23-01-2022,52.8049
22-07-2022,1065.1330
18-01-2023,103.2575
17-07-2023,2258.6384
13-01-2024,209.4553
11-07-2024,66.9170
